## Notebook Magic

In [ ]:
%matplotlib inline
%load_ext autoreload

## Imports

In [ ]:
import os
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_curve

from sklearn.linear_model import LogisticRegression, Lasso

from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from scipy.stats import randint

import itertools

from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.width = 0

In [ ]:
df = pd.read_parquet("s3://iefp-unemployment/modelling/modelling.parquet")

In [ ]:
df.shape
display(df.head(10))

In [ ]:
# Intervention feature preparation

interv_cols = [col for col in df.columns if "i_" in col]
X = df[interv_cols].copy()

# Strip col names
X.columns = [col.replace("i_", "") for col in X.columns]

# Make df boolean
X = (X.notna()).astype('int')

# Filter for frequent interventions
frequent_i = X.mean()[X.mean() > 0.01].index.tolist()
X = X[frequent_i]

In [ ]:
# Demographics prep

dems = ["d_age", "d_gender", "d_civil_status", "d_rsi", "d_desired_work_time", "d_desired_contract",
       "d_school_qualification", "d_college_qualification", "d_disabled", "d_subsidy", "d_previous_job_sector",
       "d_desired_job_sector", "d_previous_job_experience"]

X[dems] = df[dems]

In [ ]:
display(X.head(10))

In [ ]:
X.info()

In [ ]:
# Dealing with missing values
# Leave None type as feature in college_qual
# Fill NAs with 0 for school qualification!!!
# Fill NAs with 0 for previous job experience

X['d_school_qualification'] = X['d_school_qualification'].fillna(0)
X['d_previous_job_experience'] = X['d_previous_job_experience'].fillna(0)


In [ ]:
X.info()

In [ ]:
# Create temporal features

# Encode temporal variables as strings

X["register_month"] = df.register_date.dt.month.astype(str)
X["register_year"] = df.register_date.dt.year.astype(str)

# Convert categorical to dummies 

X = pd.get_dummies(X, drop_first=True, dummy_na=True)

display(X.head(10))

In [ ]:
# Set output variable

Y = df["success"]

In [ ]:
X.columns.tolist()

In [ ]:
# Test/Train split


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

In [ ]:

# Prepare Random Forest pipeline with scaling (for Age and job experience)

rf__scale_pipeline = Pipeline([
    ('scale', MinMaxScaler()),
    ('rf', RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=0))
])


In [ ]:
# Grid search

big_param_grid = [
        {'rf__n_estimators': [1,10,100,500,1000,2500],
         'rf__max_depth': [3,5,10,20,50,100],
         'rf__max_features': ['sqrt','log2'],
         'rf__min_samples_split': [2,5,10,20],
         'rf__n_jobs': [-1]
        },
    ]

med_param_grid = [
        {'rf__n_estimators': [1,10,100,1000],
         'rf__max_depth': [1,5,10,20],
         'rf__max_features': ['sqrt','log2'],
         'rf__min_samples_split': [2,5,10],
         'rf__n_jobs': [-1]
        },
    ]

param_grid = [
        {'rf__n_estimators': [500],
         'rf__max_depth': [2, 3]
        },
    ]


rf_grid_search = GridSearchCV(rf__scale_pipeline, big_param_grid, cv=3, refit=True)

rf_grid_search.fit(X_train, y_train)

rf_final_model = rf_grid_search.best_estimator_

print(confusion_matrix(y_test, rf_final_model.predict(X_test)))

print(accuracy_score(y_test, rf_final_model.predict(X_test)))

f = open("results.txt", "a")
f.write(str(rf_final_model) + "\n")
f.write(str(confusion_matrix(y_test, rf_final_model.predict(X_test))) + "\n")
f.write(str(accuracy_score(y_test, rf_final_model.predict(X_test))) + "\n")
f.write("\n")
f.close()

In [ ]:
# Permutations

print(rf_final_model.predict(X_test.head(10)))
print(y_test.head(10))

In [ ]:
combinations = [c for c in itertools.product([0, 1], repeat=16) if sum(c) <= 5]

In [ ]:
combo_df = pd.DataFrame(combinations)

In [ ]:
combo_df.shape

In [ ]:
user_data = X_test.head(1)

In [ ]:
user_data

In [ ]:
user_data_fixed = user_data.drop(user_data.iloc[:,0:16], axis=1)

In [ ]:
user_data_fixed

In [ ]:
user_data_fixed.shape

In [ ]:
user_fixed_df = pd.DataFrame(np.tile(user_data_fixed.values, len(combo_df.index)).reshape(-1,len(user_data_fixed.columns)), 
                   columns=user_data_fixed.columns)

In [ ]:
user_fixed_df.shape

In [ ]:
predict_df = combo_df.join(user_fixed_df)

In [ ]:
predict_df.shape

In [ ]:
probabilities = pd.DataFrame(rf_final_model.predict_proba(predict_df))

In [ ]:
probabilities.columns = ["unsuccessful", "successful"]

In [ ]:
results_df = predict_df.join(probabilities)

In [ ]:
results_df.iloc[:,0:16].columns = ['i_job_search_techn',
 'i_train_active_life',
 'i_tutoring_in_individual_job_search',
 'i_professional_internships',
 'i_internship_job',
 'i_employment-insertion_contract',
 'i_employment_contract_insertion',
 'i_information_job_eval_orientat',
 'i_collective_orientation_session',
 'i_tutoring_in_collective_job_search',
 'i_efa_s3_type_a',
 'i_modular',
 'i_efa_n3_vocational_train',
 'i_assertive_communication_job_search_techn',
 'i_entrepreneurship_skills_job_search_techn',
 'i_train_external_entities']

In [ ]:
display(results_df.sort_values(by=['successful'], ascending=False))

In [ ]:
# Prepare LogReg pipeline

logreg_pipeline = Pipeline([
    ('logreg', LogisticRegression(penalty='l1', C=1e5)),
])


In [ ]:
# Grid search

param_grid = [
        {'logreg__penalty': ['l1','l2'],
         'logreg__C': [0.00001,0.0001,0.001,0.01,0.1,1,10]
        },
    ]

logreg_grid_search = GridSearchCV(logreg_pipeline, param_grid, cv=3, refit=True)

logreg_grid_search.fit(X_train, y_train)

logreg_final_model = logreg_grid_search.best_estimator_

print(confusion_matrix(y_test, logreg_final_model.predict(X_test)))

print(accuracy_score(y_test, logreg_final_model.predict(X_test)))

In [ ]:
# Prepare Gradient boost pipeline

gboost_pipeline = Pipeline([
    ('gboost', GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=10)),
])

In [ ]:
# Grid search

param_grid = [
        {'gboost__n_estimators': [1,10,100,1000,10000],
         'gboost__learning_rate' : [0.001,0.01,0.05,0.1,0.5],
         'gboost__subsample' : [0.1,0.5,1.0],
         'gboost__max_depth': [1,3,5,10,20,50,100]
        },
    ]

gboost_grid_search = GridSearchCV(gboost_pipeline, param_grid, cv=3, refit=True)

gboost_grid_search.fit(X_train, y_train)

gboost_final_model = gboost_grid_search.best_estimator_

print(confusion_matrix(y_test, gboost_final_model.predict(X_test)))

print(accuracy_score(y_test, gboost_final_model.predict(X_test)))

In [ ]:
# Precision / Recall graph


def plot_precision_recall(y_test, y_pred, y_prob):
    average_precision = average_precision_score(y_test, y_pred)
    precision, recall, _ = precision_recall_curve(y_test, y_prob)
    print("{} Precision at {} Recall".format(precision[recall>0.8].max(), 0.8))

    step_kwargs = ({'step': 'post'})
    plt.step(recall, precision, color='b', alpha=0.2, where='post')
    plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(average_precision))

y_pred = pipeline.predict(X_test)
y_prob = pipeline.predict_proba(X_test)[:,1]
plot_precision_recall(y_test, y_pred, y_prob)

In [ ]:
# Roc curve

fpr, tpr, threshold = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
# Feature importance

pd.Series(rf.feature_importances_, X_train.columns).sort_values(ascending=False).head(20)

In [ ]:
# Grid search script dump:

import os
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import pickle

from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_curve
from sklearn.metrics import f1_score
from sklearn import svm

from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from scipy.stats import randint

import itertools

# evaluation

def evaluate(y_test, y_pred, y_prob):
    results = "\n---"
    results = results + str(confusion_matrix(y_test, y_pred)) + "\n---" + "Accuracy: {:.2f}%".format(100 * accuracy_score(y_test, y_pred)) + "\n---"
        
    results = results + str(average_precision_score(y_test, y_pred))
    precision, recall, _ = precision_recall_curve(y_test, y_prob)
    results = results + "{:.2f}% Precision at {}% Recall".format(100 * precision[recall>0.8].max(), 80) + "\n"
    results = results + "Precision: " + str(precision) + "\n" + "Recall: " + str(recall)
    return results


print("running...")

df = pd.read_parquet("s3://iefp-unemployment/modelling/modelling.parquet")


# modelling table construction

def transform_interventions(df: pd.DataFrame, interventions: list):
    df = df[interventions]
    df = (df.notna()).astype('int')
    
    return df


def construct_table(df): 
    # Filter for frequent interventions
    # WARNING: data leakage
    # frequent_i = df_model.mean()[df_model.mean() > 0.01].index.tolist()
    # df_model = df_model[frequent_i]
    
    
    interv_cols = [col for col in df.columns if "i_" in col]
    df_model = transform_interventions(df, interv_cols)
    
    dems = [
    "d_age",
    "d_gender",
    "d_civil_status",
    "d_rsi",
    "d_desired_work_time",
    "d_desired_contract",
    "d_disabled",
    "d_nationality",
    "d_desired_job_sector",
    "d_previous_job_sector",
    "d_school_qualification", 
    "d_college_qualification",
    "d_subsidy",
    "d_previous_job_experience",
    "register_reason",
    "d_parish",
    "d_professional_training"
    ]
    
    df_model[dems] = df[dems]
    
    # Dealing with missing values
    # Leave None type as feature in college_qual
    # Fill NAs with 0 for school qualification!!!
    # Fill NAs with 0 for previous job experience

    df_model['d_school_qualification'] = df_model['d_school_qualification'].fillna(0)
    df_model['d_previous_job_experience'] = df_model['d_previous_job_experience'].fillna(0)

    # Seasonal features
    # Encode temporal variables as strings to enable dummy var creation
    df_model["register_month"] = df.register_date.dt.month.astype(str)
    df_model["register_year"] = df.register_date.dt.year.astype(str)

    
    df_model = pd.get_dummies(df_model, drop_first=True, dummy_na=True)

    # Output
    df_model["ttj_sub_12"] = (df["journey_length"] < 365) & (df["success"] == True)
    
    return df_model

def run_grid_pipe(data, pipeline, gs_params, filename):
    
    # Get holdout set

    data["exit_date"] = df["exit_date"]
    df_hold_out = data[data["exit_date"] >= "2019-01-01"].drop("exit_date", axis=1)

    # Get training set

    data["exit_date"] = df["exit_date"]
    df_train = data[data["exit_date"] <= "2019-01-01"].drop("exit_date", axis=1)

    y = df_train["ttj_sub_12"]
    X = df_train.drop("ttj_sub_12", axis=1)
    
    grid_search = GridSearchCV(pipeline, gs_params, cv=5, refit=True, scoring="average_precision")

    grid_search.fit(X, y)

    final_model = grid_search.best_estimator_

    y_pred = final_model.predict(X)
    y_prob = final_model.predict_proba(X)[:,1]

    training_evaluation = evaluate(y, y_pred, y_prob)

    # Train on all data and then test on holdout set

    final_model.fit(X, y)

    y_hold = df_hold_out["ttj_sub_12"]
    X_hold = df_hold_out.drop("ttj_sub_12", axis=1)

    y_test_pred = final_model.predict(X_hold)
    y_test_prob = final_model.predict_proba(X_hold)[:,1]

    test_evaluation = evaluate(y_hold, y_test_pred, y_test_prob)
    

    f = open("results.txt", "a")
    f.write("\n" + filename + "\n" + str(final_model) + "\n")
    f.write("Evaluation of model on training data: \n")
    f.write(training_evaluation)
    f.write("Evaluation of model on test data: \n")
    f.write(test_evaluation)
    f.write(str(confusion_matrix(y_hold, final_model.predict(X_hold))) + "\n")
    f.write(str(accuracy_score(y_hold, final_model.predict(X_hold))) + "\n")
    f.write("\n")
    f.close()

    # save the model to disk

    pickle.dump(final_model, open(filename, 'wb'))


##############################################

df_model = construct_table(df)

##############################################


# Prepare Random Forest pipeline with scaling (for Age and job experience)

rf__scale_pipeline = Pipeline([
    ('scale', MinMaxScaler()),
    ('rf', RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=0, class_weight="balanced"))
])


# RF Grid search

big_param_grid = [
        {'rf__n_estimators': [1,10,100,500,1000,2500],
         'rf__max_depth': [3,5,10,20,50,100],
         'rf__max_features': ['sqrt','log2'],
         'rf__min_samples_split': [2,5,10,20],
         'rf__n_jobs': [-1]
        },
    ]

med_param_grid = [
        {'rf__n_estimators': [1,10,100,1000],
         'rf__max_depth': [1,5,10,20],
         'rf__max_features': ['sqrt','log2'],
         'rf__min_samples_split': [2,5,10],
         'rf__n_jobs': [-1]
        },
    ]

param_grid = [
        {'rf__n_estimators': [500],
         'rf__max_depth': [2, 3]
        },
    ]

# Prepare LogReg pipeline

logreg_pipeline = Pipeline([
    ('scale', MinMaxScaler()),
    ('logreg', LogisticRegression(penalty='l1', C=1e5, class_weight="balanced", n_jobs=-1)),
])

# LR Grid search

lg_param_grid = [
        {'logreg__penalty': ['l1','l2'],
         'logreg__C': [0.00001,0.0001,0.001, 0.01,0.1,1,10]
        },
    ]

# Prepare Gradient boost pipeline

gboost_pipeline = Pipeline([
    ('scale', MinMaxScaler()),
    ('gboost', GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=10)),
])


# Gradient Boosting Trees Grid search

gb_param_grid = [
        {'gboost__n_estimators': [1,10,100,1000,10000],
         'gboost__learning_rate' : [0.001,0.01,0.05,0.1,0.5],
         'gboost__subsample' : [0.1,0.5,1.0],
         'gboost__max_depth': [1,3,5,10,20,50,100]
        },
    ]

# Prepare adaboost pipeline

adaboost_pipeline = Pipeline([
    ('scale', MinMaxScaler()),
    ('adaboost', AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), algorithm="SAMME", n_estimators=200))
])


# Ada grid search

ada_param_grid = [
        {'algorithm': ['SAMME', 'SAMME.R'],
         'n_estimators': [1,10,100,500,1000,10000]
        },
    ]

# Prepare SVM pipeline

svm_pipeline = Pipeline([
    ('scale', MinMaxScaler()),
    ('svm', svm.SVC(kernel='linear', probability=True, random_state=0, class_weight="balanced"))
])

svm_param_grid = [
        {'C' :[0.00001,0.0001,0.001,0.01,0.1,1,10],
         'gamma':[1,0.1,0.001,0.0001],
         'kernel':['linear', 'rbf', 'polynomial', 'sigmoid']
        },
    ]

# Run pipelines

run_grid_pipe(df_model, rf__scale_pipeline, param_grid, "rf_test.sav")
run_grid_pipe(df_model, logreg_pipeline, lg_param_grid, "lg_test.sav")
run_grid_pipe(df_model, gboost_pipeline, gb_param_grid, "gb_test.sav")
run_grid_pipe(df_model, adaboost_pipeline, ada_param_grid, "ada_test.sav")
run_grid_pipe(df_model, svm_pipeline, svm_param_grid, "svm_test.sav")



print("Done!")
